In [ ]:
#| output: false
using CSV
using DataFrames
using DataFramesMeta
using Distributions
using Interpolations
using Plots
using StatsPlots
using Unitful

Plots.default(; margin=6Plots.mm)

include("depthdamage.jl")

In [ ]:
#| output: false
include("depthdamage.jl")

# Single Year Function


In [ ]:
haz_fl_dept = CSV.read("data/haz_fl_dept.csv", DataFrame) # read in the file
desc = "one story, Contents, fresh water, short duration"
row = @rsubset(haz_fl_dept, :Description == desc)[1, :] # select the row I want
dd = DepthDamageData(row) # extract the depth-damage data
damage_fn = get_depth_damage_function(dd.depths, dd.damages) # get the depth-damage function

In [ ]:
function single_year_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh)
    
    # calculate the expected damages
    gauge_dist = GeneralizedExtremeValue(5, 1, 0.1)
    offset = 4.31
   
    house_dist = GeneralizedExtremeValue(gauge_dist.μ - offset, gauge_dist.σ, gauge_dist.ξ)
    samples = rand(house_dist, 100_000) .* 1u"ft"
    
    damages = damage_fn.(samples)
    expected_damages_pct = mean(damages)
        
    house_structure_value = 475_000 # estimated value of Katie's Seafood House on Zillow
    expected_damages_usd = house_structure_value * expected_damages_pct / 100
   
    c_dmg = expected_damages_usd

    # calculate the cost of elevating
    heights = uconvert.(u"ft", (0u"ft"):(1u"inch"):(10u"ft")) 
plot(
    heights,
    elevation_cost.(heights, house_area);
    xlabel="How High to Elevate",
    ylabel="Cost (USD)",
    label="$(house_area)",
    tiitle="Cost of Elevating a House",
)
    house_area = 1000u"ft^2"
    elevation_cost = get_elevation_cost_function()
    
    c_constr = elevation_cost

    # return the total cost and benefit
    return -c_constr - c_dmg
end

# NPV Function


In [ ]:
function npv_cost_benefit(flood_dist, damage_fn, elevation_cost, house_area, house_value, Δh, T, discount_rate)
    # calculate the costs and benefits for each year, and then discount
    # see above!
    return npv
end

In [ ]:
annual_damages = [expected_damages_usd for _ in 1:10] # annual expected damages
discount_rate = 0.05
npv = sum(annual_damages .* (1 - discount_rate) .^ (0:9))

# One SOW, several actions

# Sensitivity test

# Discussion
1. What do you notice about the NPV for different actions?
2. What do you notice about the sensitivity test?
3. What are some limitations of this analysis?
    * What things are missing from this analysis that you think are important?
    * How might they affect the results?
    * What are some ways you might address these limitations?